## 2.8. 파이프라인

머신러닝 작업은 데이터 전처리, 모델 불러오기, 예측, 후처리 등 여러 단계로 이루어집니다. 이러한 단계를 연결하여 하나의 작업 흐름으로 만들어주는 것이 파이프라인입니다. 파이프라인을 사용하면 여러 단계를 연결하여 전체 작업을 수행할 수 있습니다. 파이프라인을 사용하면 코드를 간결하게 작성할 수 있고, 전체 작업을 한 번에 실행할 수 있습니다.


### 2.8.1. 파이프라인 종류와 예시

**허깅페이스 파이프라인**

* **자연어 처리 파이프라인**: 토크나이저, 모델, 후처리로 구성되며, 텍스트 생성, 요약, 번역, 질문 답변 등의 과제에 사용된다.
* **컴퓨터비전 파이프라인**: 이미지 프로세서, 모델, 후처리로 이루어져 있으며 이미지 분류, 객체 탐지, 세그멘테이션 등의 과제를 수행한다.
* **오디오 파이프라인**: 오디오 전처리, 모델, 후처리로 구성되며, 음성 인식, 음성 생성, 감정 분석 등의 과제에 사용된다.
* **멀티모달 파이프라인**: 텍스트, 이미지, 오디오 등 다양한 모달리티를 다루는 파이프라인이다.

각 파이프라인은 구성 요소를 자동으로 연결하여 전체 작업을 수행할 수 있습니다. 파이프라인을 사용하면 코드를 간결하게 작성할 수 있고, 전체 작업을 한 번에 실행할 수 있습니다.



In [15]:
# 자연어 처리 파이프라인 예시

import torch
import pandas as pd
import torch.nn.functional as F
from transformers import pipeline

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 텍스트 분류 파이프라인 설정
pipe = pipeline(
  task="text-classification",
  model="cardiffnlp/twitter-roberta-base-sentiment-latest",
  device=device
)

# 입력 텍스트
raw_text = [
  "I love you.",
  "I hate you.",
  "I meet with you.",
  "I don't meet with you.",
  "I don't know you.",
  "I don't like you.",
  "I don't love you.",
  "I don't hate you.",
  "I don't meet with you.",
  "I don't know you.",
]

# 토크나이저를 사용하여 입력 텍스트를 텐서로 변환
inputs = pipe.tokenizer(raw_text, return_tensors="pt", padding=True)

# 모델을 사용하여 예측 수행
outputs = pipe.model(inputs["input_ids"].to(device))

# 소프트맥스 함수를 사용하여 확률 계산
probabilities = F.softmax(outputs.logits, dim=1)

# 예측 결과 저장
predictions = []

for prob in probabilities:
  max_id = torch.argmax(prob).item()
  class_name = pipe.model.config.id2label[max_id]
  score = prob[max_id].item()
  predictions.append((class_name, score))

# 원본 텍스트 처리
prediction = pipe(raw_text)

print("- 원본 텍스트:")
print(raw_text)

print("- input_ids:")
print(inputs["input_ids"])

print("- logits:")
print(outputs.logits)

# 예측 결과를 데이터프레임으로 변환
df = pd.DataFrame(predictions, columns=["class", "score"])
df["text"] = raw_text
df


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


- 원본 텍스트:
['I love you.', 'I hate you.', 'I meet with you.', "I don't meet with you.", "I don't know you.", "I don't like you.", "I don't love you.", "I don't hate you.", "I don't meet with you.", "I don't know you."]
- input_ids:
tensor([[   0,  100,  657,   47,    4,    2,    1,    1,    1],
        [   0,  100, 4157,   47,    4,    2,    1,    1,    1],
        [   0,  100,  972,   19,   47,    4,    2,    1,    1],
        [   0,  100,  218,   75,  972,   19,   47,    4,    2],
        [   0,  100,  218,   75,  216,   47,    4,    2,    1],
        [   0,  100,  218,   75,  101,   47,    4,    2,    1],
        [   0,  100,  218,   75,  657,   47,    4,    2,    1],
        [   0,  100,  218,   75, 4157,   47,    4,    2,    1],
        [   0,  100,  218,   75,  972,   19,   47,    4,    2],
        [   0,  100,  218,   75,  216,   47,    4,    2,    1]])
- logits:
tensor([[-1.8678, -0.2180,  2.0666],
        [ 1.8566, -0.1212, -2.0510],
        [-1.8801,  1.5437, -0.0947],
       

,class,score,text
0,positive,0.891757,I love you.
1,negative,0.863207,I hate you.
2,neutral,0.815075,I meet with you.
3,neutral,0.718518,I don't meet with you.
4,neutral,0.690650,I don't know you.
5,negative,0.848319,I don't like you.
6,negative,0.722096,I don't love you.
7,neutral,0.558379,I don't hate you.
8,neutral,0.718518,I don't meet with you.
9,neutral,0.690650,I don't know you.


In [17]:
# 자연어 처리 파이프라인 예시
import torch
import pandas as pd
from transformers import pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 텍스트 분류 파이프라인 설정
pipe = pipeline(
  task="text-classification",
  model="cardiffnlp/twitter-roberta-base-sentiment-latest",
  device=device
)

# 입력 텍스트
raw_text = [
  "I love you.",
  "I hate you.",
  "I meet with you.",
  "I don't meet with you.",
  "I don't know you.",
  "I don't like you.",
  "I don't love you.",
  "I don't hate you.",
  "I don't meet with you.",
  "I don't know you.",
]

prediction = pipe(raw_text)
df = pd.DataFrame(prediction)
df["text"] = raw_text
df

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,label,score,text
0,positive,0.928684,I love you.
1,negative,0.884135,I hate you.
2,neutral,0.844113,I meet with you.
3,neutral,0.718518,I don't meet with you.
4,neutral,0.706533,I don't know you.
5,negative,0.863184,I don't like you.
6,negative,0.751521,I don't love you.
7,neutral,0.586926,I don't hate you.
8,neutral,0.718518,I don't meet with you.
9,neutral,0.706533,I don't know you.


### 2.8.2. 파이프라인 함수

* **task**: 수행하려는 과제를 설정한다. 텍스트 생성, 이미지 분류, 음성 인식 등의 과제를 설정할 수 있다.
* **model**: 파이프라인에서 사용할 모델을 지정한다. 모델의 이름이나 경로 또는 직접 `PreTrainedModel` 객체를 지정할 수 있다. 지정하지 않으면 지맘대로 한다.
* **config**: 모델 구성 파일의 이름이나 경로 또는 `PretrainedConfig` 객체를 지정한다. 지정하지 않으면 지맘대로 한다.
* **tokenizer**: 파이프라인에 사용할 토크나이저를 지정한다. 토크나이저 이름이나 경로 또는 `PreTrainedTokenizer` 객체를 지정할 수 있다. 지정하지 않으면 지맘대로 한다.
* **feature_extractor**: 특징 추출기를 지정한다. 특징 추출기의 이름이나 경로 또는 `PreTrainedFeatureExtractor` 객체를 지정할 수 있다. 지정하지 않으면 지맘대로 한다.
* **image_processor**: 이미지 프로세서를 지정한다. 이미지 프로세서의 이름이나 경로 또는 `BaseImageProcessor` 객체를 지정할 수 있다. 지정하지 않으면 지맘대로 하는지는 설명하지 않고 있다.
* **framework**: 파이프라인에서 사용할 딥러닝 프레임워크를 지정한다. `pt` 또는 `tf`를 지정할 수 있다. 지정하지 않으면 지맘대로 한다. 큰일 날 수 있다.
* **revision**: 모델 허브에서 모델을 다운로드할 때 사용할 버전을 지정한다. 브렌치 이름, 태그 이름, 커밋 ID를 입력할 수 있다.
* **use_fast**: 빠른 토크나이저를 사용할 지 여부를 지정한다.
* **token**: 모델 허브에서 모델을 다운로드할 때 사용할 액서스 토큰을 지정한다.
* **device**: 디바이스를 지정한다. `cpu` 또는 `cuda`를 지정할 수 있다. GPU번호를 지정할 수 있다.
* **device_map**: 모델의 각 모듈을 어떤 장치에서 불러올지 지정한다. `auto`로 설정하면 가장 최적화된 device_map이 자동으로 계산된다. `device`와 `device_map`을 동시에 사용하면 충돌이 발생할 수 있다.
* **torch_dtype**: 모델의 가중치를 불러올때 사용할 정밀도(torch.float16, torch.bfloat16)를 지정한다.
* **trust_remote_code**: 사용자 정의 모델링, 구성, 토크나이저 또는 파이프라인 파일의 코드를 신뢰할지 여부를 지정한다.
* **model_kwargs**: 모델을 생성할 때 추가로 전달할 매개변수를 딕셔너리 형태로 지정한다.
* **pipeline_class**: 사용자 정의 파이프라인 클래스를 지정한다.


In [27]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Initialize the model and tokenizer
model_name = "openai-community/gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize the pipeline
pipe = pipeline(
  task="text-generation",
  model=model,
  tokenizer=tokenizer,
  device=device.index if device.type == "cuda" else -1
)

# Input text
inputs = "I am learning about tokenizers."

# Generate text
outputs = pipe(inputs)

print(outputs)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'I am learning about tokenizers. It became apparent that if tokenizers are implemented in an industry as lucrative but costly as crypto they will also be deployed on a more profitable platform. In the meantime we are studying what we could do to improve the overall'}]


In [32]:
# 컴퓨터비전 파이프라인
import torch
import pandas as pd
from datasets import load_dataset
from transformers import pipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline(
  task="image-classification",
  model="google/vit-base-patch16-224",
  device=device,
  model_kwargs={
    "hidden_act": "gelu", # 활성화 함수
  }
)

# Load the dataset
dataset = load_dataset("huggingface/cats-image")
inputs = dataset['test']['image'][0]
outputs = pipe(inputs)
pd.DataFrame(outputs)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


,label,score
0,Egyptian cat,0.937441
1,"tabby, tabby cat",0.038443
2,tiger cat,0.014411
3,"lynx, catamount",0.003274
4,"Siamese cat, Siamese",0.000680
